<a href="https://colab.research.google.com/github/CodeMonkey01/DataMiningI/blob/main/ANN/Option_A/ANN_with_BERT_add_BERT_seperator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANN with BERT
In this notebook I tried to solve the classification model with an ANN based on pretrained BERT layers.

## TODO:
- [ ] Stemming
- [ ] Remove Stop Words
- [x] BERT Seperator

## Results
### Result with no hyperparamater optimization
accuracy: 0.9297 - precision: 0.9260 - recall: 0.9341

### Result with hyperparamater optimization
accuracy: 0.9290 - precision: 0.9125 - recall: 0.9494

### Result with hyperparamater optimization and cross val
Accuracy: 0.9284850060939789
Precision: 0.9364166557788849
Recall: 0.9201099872589111

## Model
Link to the model: https://drive.google.com/file/d/1-DcCa5FowzP-bHRPYxY7t0MAwo5gnBjN/view?usp=sharing, https://drive.google.com/file/d/1-IqI6Xf3s4oU5Eer6D46Hdn3405qw8uD/view?usp=sharing 

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd '/content/drive/MyDrive/'

    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)
except ImportError as e:
    pass

Mounted at /content/drive/
/content/drive/MyDrive
Tue May 24 06:10:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [ ]:
!pip install tensorflow_text
!pip install tensorflow_hub
!pip install transformers
#!pip install scikeras[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 5.0 MB/s 
     - 677.8 MB 405 kB/s
     |████████████████████████████████| 5.8 MB 65.5 MB/s 
     |████████████████████████████████| 1.6 MB 67.6 MB/s 
     |████████████████████████████████| 438 kB 71.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfu

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/Data Mining/dataset.txt')
df_raw.describe()

In [5]:
# todo --> take (random) sample to speed up training
df = df_raw.sample(5_000)

# Check for imbalance
The dataset is equally balanced. Therefore, we do not need to rebalance the dataset.

In [6]:
df["humor"].value_counts()

False    100000
True     100000
Name: humor, dtype: int64

# Preprocessing

In [7]:
# Transform class from Boolean to integer value
df['class']=df['humor'].apply(lambda x: 1 if x==True else 0)

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

MAX_LEN = 128
#pad_to_max_length=True,
df['bert_preprocessed']=df['text'].apply(lambda x: " ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(x, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
df.head()

,text,humor,class,bert_preprocessed
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,0,[CLS] joe bid ##en rules out 2020 bid : ' guys...
1,Watch: darvish gave hitter whiplash with slow ...,False,0,[CLS] watch : dar ##vish gave hitter whip ##la...
2,What do you call a turtle without its shell? d...,True,1,[CLS] what do you call a turtle without its sh...
3,5 reasons the 2016 election feels so personal,False,0,[CLS] 5 reasons the 2016 election feels so per...
4,"Pasco police shot mexican migrant from behind,...",False,0,[CLS] pas ##co police shot mexican migrant fro...


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['bert_preprocessed'], df['class'], test_size=0.4)
X = df['bert_preprocessed']
Y = df['class']

# BERT

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

## Test embedding
Test word embedding from pretrained BERT model with a real sentence from dataset.

In [ ]:
#test_sentence = df["bert_preprocessed"][11704]
#print("Test sentence:")
#print(test_sentence)
#print("Test sentence (word embedding):")
#print(get_sentence_embeding([test_sentence]))

# Build model

In [14]:
def build_model() -> tf.keras.Model:
    # Bert layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

    # Softmax is shit for this binary classification problem. Event though the literature suggest this activation function for binary class problem
    #l = tf.keras.layers.Dense(1, activation='softmax', name="output")(l) 

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[text_input], outputs = [l])

    #model.summary()

    return model

In [13]:
import numpy as np
import itertools
EPOCHS = 5

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]


lr_values = [0.0044] # np.arange(1e-3, 1e-2, 0.0002)
epsilon_values = [1e-5]#[1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
batch_values = [256]#range(32, 512, 16)

values = list(itertools.product(lr_values, epsilon_values, batch_values))

#print(f"Combinations: {len(values)}")
#for lr, ep, batch in values:
#    model: tf.keras.Model = build_model()

#    print(f"Try adam learning rate of: {lr} and e: {ep} and batch size: {batch}")

 #   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr, epsilon=ep),
       #           loss=tf.keras.losses.BinaryCrossentropy(),
        #          
         #         metrics=METRICS)
    
  #  model.fit(X_train, y_train, epochs=EPOCHS, batch_size=batch)
 #   model.evaluate(X_test, y_test)

Add epoch optimization

https://stackoverflow.com/questions/47421951/keras-optimal-epoch-selection

In [16]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

ADAM_LEARNING_RATE = 0.0044
ADAM_EPSILON = 1e-05

EPOCH = 50

BATCH_SIZE = 64

PATIENCE = 2

base_path = "/content/drive/MyDrive/Data Mining/bert/"

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

def train_evaluate(model, run, x_train, y_train, x_test, y_test):
    STAMP = 'model_run_%.2f'%(run)
    early_stopping = EarlyStopping(monitor='val_loss', patience=PATIENCE)
    bst_model_path = base_path + STAMP + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=ADAM_LEARNING_RATE, epsilon=ADAM_EPSILON),
                  loss=tf.keras.losses.BinaryCrossentropy(),                  
                  metrics=METRICS)
    

    X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, test_size=0.2)

    hist = model.fit(X_tra, y_tra, validation_data=(X_val, y_val), epochs=EPOCH, batch_size=BATCH_SIZE, shuffle=True, callbacks=[early_stopping, model_checkpoint])

    return model.evaluate(x_test, y_test)

kFold = StratifiedKFold(n_splits=4)
accuracy_list = []
precision_list = []
recall_list = []
i = 1
for train, test in kFold.split(X, Y):
    model = build_model()
    _, accuracy, precision, recall = train_evaluate(model, i, X[train], Y[train], X[test], Y[test])
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    i = i + 1

print(f"Accuracy: {sum(accuracy_list) / len(accuracy_list)}")
print(f"Precision: {sum(precision_list) / len(precision_list)}")
print(f"Recall: {sum(recall_list) / len(recall_list)}")

Epoch 1/50
1875/1875 [==============================] - 1566s 834ms/step - loss: 0.2889 - accuracy: 0.8822 - precision: 0.8737 - recall: 0.8938 - val_loss: 0.2261 - val_accuracy: 0.9139 - val_precision: 0.9454 - val_recall: 0.8777
Epoch 2/50
1875/1875 [==============================] - 1563s 834ms/step - loss: 0.2391 - accuracy: 0.9045 - precision: 0.9000 - recall: 0.9103 - val_loss: 0.1924 - val_accuracy: 0.9272 - val_precision: 0.9225 - val_recall: 0.9323
Epoch 3/50
1875/1875 [==============================] - 1560s 832ms/step - loss: 0.2345 - accuracy: 0.9069 - precision: 0.9035 - recall: 0.9112 - val_loss: 0.1866 - val_accuracy: 0.9295 - val_precision: 0.9337 - val_recall: 0.9240
Epoch 4/50
1875/1875 [==============================] - 1559s 831ms/step - loss: 0.2330 - accuracy: 0.9078 - precision: 0.9050 - recall: 0.9114 - val_loss: 0.2011 - val_accuracy: 0.9209 - val_precision: 0.8879 - val_recall: 0.9627
Epoch 5/50
1563/1563 [==============================] - 495s 317ms/step - lo

# Evaluate epsilon
We tried out different epsilon values.

Each step was processed in 5 epochs with a sample size of 20k rows. 

| Epsilon | Accuracy - Precision - Recall                         |
|---------|-------------------------------------------------------|
| 1e-07   | accuracy: 0.8958 - precision: 0.8899 - recall: 0.9038 |
| 1e-06   | accuracy: 0.8955 - precision: 0.8881 - recall: 0.9055 |
| 1e-05   | accuracy: 0.8987 - precision: 0.8923 - recall: 0.9072 |
| 0.0001  | accuracy: 0.8963 - precision: 0.8899 - recall: 0.9048 |
| 0.001   | accuracy: 0.8956 - precision: 0.8888 - recall: 0.9047 |
| 0.01    | accuracy: 0.8903 - precision: 0.8839 - recall: 0.8992 |
| 0.1     | accuracy: 0.8731 - precision: 0.8614 - recall: 0.8897 |
| 0.2     | accuracy: 0.8597 - precision: 0.8434 - recall: 0.8842 |
| 0.3     | accuracy: 0.8578 - precision: 0.8418 - recall: 0.8819 |
| 0.4     | accuracy: 0.8472 - precision: 0.8294 - recall: 0.8747 |
| 0.5     | accuracy: 0.8369 - precision: 0.8193 - recall: 0.8652 |
| 0.6     | accuracy: 0.8275 - precision: 0.8081 - recall: 0.8597 |
| 0.7     | accuracy: 0.8191 - precision: 0.7987 - recall: 0.8541 |
| 0.8     | accuracy: 0.8219 - precision: 0.8030 - recall: 0.8539 |
| 0.9     | accuracy: 0.8117 - precision: 0.7949 - recall: 0.8409 |
| 1       | accuracy: 0.7956 - precision: 0.7732 - recall: 0.8376 |

> the lr was 0.0038000000000000013.

The epsilon of **1e-05** reaches the highest accuracy, precision and recall. 

In [17]:
_Ä'

SyntaxError: ignored

# Evaluate learning rate

Based on the identified epsilon of 1e-05 we tried out different learning rates.

Each step was processed in 5 epochs with a sample size of 20k rows. 

| Learning rate |     Accuracy - Precision - Recall                     |
|---------------|-------------------------------------------------------|
| 0.001         | accuracy: 0.8917 - precision: 0.8801 - recall: 0.9073 |
| 0.0012        | accuracy: 0.8852 - precision: 0.8766 - recall: 0.8970 |
| 0.0014        | accuracy: 0.8917 - precision: 0.8822 - recall: 0.9045 |
| 0.0016        | accuracy: 0.8906 - precision: 0.8821 - recall: 0.9022 |
| 0.0018        | accuracy: 0.8932 - precision: 0.8829 - recall: 0.9072 |
| 0.002         | accuracy: 0.8887 - precision: 0.8805 - recall: 0.8998 |
| 0.0022        | accuracy: 0.8917 - precision: 0.8827 - recall: 0.9040 |
| 0.0024        | accuracy: 0.8947 - precision: 0.8866 - recall: 0.9057 |
| 0.0026        | accuracy: 0.8939 - precision: 0.8863 - recall: 0.9042 |
| 0.0028        | accuracy: 0.8952 - precision: 0.8890 - recall: 0.9035 |
| 0.0030        | accuracy: 0.8920 - precision: 0.8854 - recall: 0.9010 |
| 0.0032        | accuracy: 0.8921 - precision: 0.8849 - recall: 0.9018 |
| 0.0034        | accuracy: 0.8931 - precision: 0.8875 - recall: 0.9007 |
| 0.0036        | accuracy: 0.8932 - precision: 0.8853 - recall: 0.9040 |
| 0.0038        | accuracy: 0.8963 - precision: 0.8910 - recall: 0.9035 |
| 0.0040        | accuracy: 0.8950 - precision: 0.8881 - recall: 0.9043 |
| 0.0042        | accuracy: 0.8910 - precision: 0.8855 - recall: 0.8985 |
| 0.0044        | accuracy: 0.8997 - precision: 0.8927 - recall: 0.9092 |
| 0.0046        | accuracy: 0.8941 - precision: 0.8875 - recall: 0.9030 |
| 0.0048        | accuracy: 0.8921 - precision: 0.8883 - recall: 0.8973 |
| 0.0050        | accuracy: 0.8962 - precision: 0.8895 - recall: 0.9052 |
| 0.0052        | accuracy: 0.8953 - precision: 0.8895 - recall: 0.9030 |
| 0.0054        | accuracy: 0.8992 - precision: 0.8931 - recall: 0.9075 |
| 0.0056        | accuracy: 0.8878 - precision: 0.8807 - recall: 0.8977 |
| 0.0058        | accuracy: 0.8903 - precision: 0.8851 - recall: 0.8975 |
| 0.0060        | accuracy: 0.8979 - precision: 0.8920 - recall: 0.9058 |
| 0.0062        | accuracy: 0.8951 - precision: 0.8886 - recall: 0.9038 |
| 0.0064        | accuracy: 0.8977 - precision: 0.8944 - recall: 0.9022 |
| 0.0066        | accuracy: 0.8940 - precision: 0.8876 - recall: 0.9027 |
| 0.0068        | accuracy: 0.8921 - precision: 0.8877 - recall: 0.8982 |
| 0.0070        | accuracy: 0.8920 - precision: 0.8863 - recall: 0.8998 |
| 0.0072        | accuracy: 0.8948 - precision: 0.8896 - recall: 0.9020 |
| 0.0074        | accuracy: 0.8953 - precision: 0.8901 - recall: 0.9025 |
| 0.0076        | accuracy: 0.8940 - precision: 0.8893 - recall: 0.9005 |
| 0.0078        | accuracy: 0.8891 - precision: 0.8851 - recall: 0.8947 |
| 0.0080        | accuracy: 0.8967 - precision: 0.8923 - recall: 0.9028 |
| 0.0082        | accuracy: 0.8922 - precision: 0.8871 - recall: 0.8992 |
| 0.0084        | accuracy: 0.8927 - precision: 0.8864 - recall: 0.9012 |
| 0.0086        | accuracy: 0.8985 - precision: 0.8930 - recall: 0.9058 |
| 0.0088        | accuracy: 0.8969 - precision: 0.8931 - recall: 0.9022 |
| 0.0090        | accuracy: 0.8909 - precision: 0.8879 - recall: 0.8952 |
| 0.0092        | accuracy: 0.8918 - precision: 0.8879 - recall: 0.8973 |
| 0.0094        | accuracy: 0.8929 - precision: 0.8899 - recall: 0.8972 |
| 0.0096        | accuracy: 0.8941 - precision: 0.8879 - recall: 0.9025 |
| 0.0098        | accuracy: 0.8930 - precision: 0.8904 - recall: 0.8967 |

> The epsilon was 1e-05.

The learning rate of **0.0044** receaches the highest accuracy, precision and recall. 

Therefore we consider this as the new learning rate. 

In [ ]:
_Ä'odel.evaluate(X_test, y_test)

In [ ]:
_Ä'model.save_weights("/content/drive/MyDrive/Data Mining/bert_seperator/weights.md5")

In [ ]:
_Ä'_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
_import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))

# Own test

In [ ]:
jokes = [
    'What’s the best thing about Switzerland? I don’t know, but the flag is a big plus.',
    'I study Business Informatics at the University of Mannheim!',
    'I invented a new word! Plagiarism!',
    'Did you hear about the mathematician who’s afraid of negative numbers? He’ll stop at nothing to avoid them.',
    'My name is Elias.'
]

jokes_processed = [" ".join(list(tokenizer.convert_ids_to_tokens(tokenizer.encode(i, add_special_tokens=True, max_length=MAX_LEN, truncation=True)))) for i in jokes]

model.predict(jokes_processed)